## Normalized Region --> NYC (Final Map)

#### NB setup

In [1]:
import pandas as pd
import json
import geopandas as gpd
import plotly.express as px
import os

In [2]:
# make display wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# set directory explicitly
os.chdir(r'C:\Users\steve\GitHub\rp-covid-migration')

In [3]:
# read in GeoJSON, convert to geopandas
subplaces = r'data/shapefiles/nyc_subplace_centroid.geojson'
subplaces = gpd.read_file(subplaces)
subplaces.head()

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geometry
0,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.321040,0.005154,47773634.0,18.44540,900946940.0,-234.0,POINT (-73.12283 41.52707)
1,2,09,009,47535,00213462,0900947535,Milford,Milford town,43,T5,G4040,720,71950,None,C,57442071.0,10218658.0,+41.2250861,-073.0611101,None,None,None,None,0.481270,0.007266,67660729.0,26.12381,900947535.0,-1743.0,POINT (-73.06185 41.22575)
2,3,09,009,58300,00213486,0900958300,Oxford,Oxford town,43,T1,G4040,720,71950,None,A,84803121.0,1531057.0,+41.4440006,-073.1479992,None,None,None,None,0.439399,0.009300,86334178.0,33.33363,900958300.0,248.0,POINT (-73.13503 41.43129)
3,4,09,009,00000,00000000,0900900000,County subdivisions not defined,County subdivisions not defined,00,Z9,G4040,None,None,None,F,0.0,599104136.0,+41.1874659,-072.8153339,None,None,None,None,1.700000,0.064300,599104136.0,231.31411,900900000.0,NaN,POINT (-72.79470 41.25792)
4,5,09,009,44560,00213454,0900944560,Madison,Madison town,43,T1,G4040,720,75700,None,A,93622105.0,1251401.0,+41.3444810,-072.6245213,None,None,None,None,0.739953,0.010206,94873506.0,36.63066,900944560.0,-218.0,POINT (-72.62809 41.34013)


#### Read in the returns file, clean up the data for viz

In [4]:
df = pd.read_csv(r'data/subplxborough/subxnyc_01-10-2020.csv')

decimals = 0 # to round to whole num in lambda func below


df = df.rename(columns = {'orig_subpl_str':'id'})
df['id'] = df['id'].str[1:] # remove the 'U' in the column

list_date = df.columns.to_list()
list_date = list_date[2:]
# melt so we have boroughs as a column.
df = pd.melt(df, id_vars=['id','date_y-m-d'], value_vars = list_date)
# rename the columns for clarity
df = df.rename(columns = {'variable':'Borough','date_y-m-d':'date'})
# add explicit borough column -- New York City, Bronx, Brooklyn, Manhattan, Queens, Staten Island
# borough_name = {'New York City':'NYC', 'Bronx':'BX', 'Brooklyn':'BK', 'Manhattan':'MN', 'Queens':'QN', 'Staten Island':'SI'}
df['Borough'] = df['Borough'].replace({'NYC':'New York City', 'BX':'Bronx', 'BK':'Brooklyn', 'MN':'Manhattan', 'QN':'Queens', 'SI':'Staten Island'})
# parse month out of dates
new = df["date"].str.split("/", n = 1, expand = True) 
df["month"]= new[0].astype(int) # convert to integer
# remove october dates --- for now, remove this step when you add entire year
df = df.drop(df[df.month > 9].index)
# convert to date
df['date'] = pd.to_datetime(df['date'])
# merge with counties
df = pd.merge(subplaces, df, left_on='GEOID', right_on='id', how = "inner")
# set to this date format
df['date'] = df["date"].dt.strftime('%m/%d/%Y')
# conversions for visualization purposes
df['value'] = df['value'].astype(float)
df['value'] = df['value'].apply(lambda x: round(x, decimals)) # round to nearest whole num
df['value'] = df['value'].astype(int) # remove decimal
#remove rows less that are 0, we don't want them to show.
df = df.loc[df['value']>=1]
df = df.rename(columns = {'NAME':'Name','date':'Date','value':'Returns'})
# sort for visualization
df = df.sort_values(by = 'Date')
# remove entire city
df = df[df['Borough'] != 'New York City']
# save this cleaned file
df.to_csv(f'data/subplxborough/norm_subplace_x_nyc.csv', index = False)

df.head()

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,Name,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geometry,id,Date,Borough,Returns,month
0,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.321040,0.005154,47773634.0,18.44540,9.009469e+08,-234.0,POINT (-73.12283 41.52707),0900946940,01/01/2020,Bronx,70,1
814516,580,36,119,50078,00979260,3611950078,New Castle,New Castle town,43,T1,G4040,None,None,None,A,59992981.0,941675.0,+41.1825336,-073.7728472,None,None,None,None,0.487228,0.006540,60934656.0,23.52687,3.611950e+09,-186.0,POINT (-73.77256 41.18422),3611950078,01/01/2020,Staten Island,33,1
815064,581,36,119,51693,00979282,3611951693,North Castle,North Castle town,43,T1,G4040,None,None,None,A,61672183.0,6755726.0,+41.1363268,-073.6905015,None,None,None,None,0.604071,0.007338,68427909.0,26.42002,3.611952e+09,-611.0,POINT (-73.69497 41.13313),3611951693,01/01/2020,Bronx,188,1
113626,93,34,017,52470,00882223,3401752470,North Bergen,North Bergen township,44,T1,G4040,None,None,None,A,13301694.0,1128246.0,+40.7941628,-074.0249470,None,None,None,None,0.282473,0.001540,14429940.0,5.57140,3.401752e+09,-2515.0,POINT (-74.02593 40.79373),3401752470,01/01/2020,Brooklyn,1074,1
815338,581,36,119,51693,00979282,3611951693,North Castle,North Castle town,43,T1,G4040,None,None,None,A,61672183.0,6755726.0,+41.1363268,-073.6905015,None,None,None,None,0.604071,0.007338,68427909.0,26.42002,3.611952e+09,-611.0,POINT (-73.69497 41.13313),3611951693,01/01/2020,Brooklyn,219,1


#### Plotly Visualization

In [ ]:
fig = px.scatter_mapbox(df,# geojson = subplaces, this geojson is not needed if you have lat longs as separate columns in df
                     lat = 'INTPTLAT', lon = 'INTPTLON',
                     size = 'Returns', # column to reference bubble sizes
                     color ='Borough', # change color based on borough
                     color_discrete_sequence = ['#CCCCFF','#1D819A','#FFE066','#63BB69','#DE7A3F'], # calling in explicit borough colors we created
                     animation_frame = 'Date',
                     animation_group = 'Borough',
                     hover_data = {'Name':True,'Date':True,'Returns':True, 'INTPTLAT':False, 'INTPTLON':False, 'Borough':False}, # specifying hover data layers
                     #font = dict(family = 'Arial', color = 'white'),
                     size_max = 60)
fig.update_layout(mapbox_zoom = 6.8,
                 mapbox_center = {'lat':40.82, 'lon':-74.25},
                 mapbox_style="carto-darkmatter",
                # size of the viz
                 width = 950,
                 height = 780, 
                 font = dict(family = 'Arial', color = 'black'),
                 title_font_color = 'white',
                # set legend params
                 legend=dict(
                      title = 'Destination',
                      title_font_color = 'white',
                      bgcolor = 'black',
                      bordercolor = 'gray', borderwidth = 1,
                      font = dict(family = 'Arial', color = 'white'),
                      yanchor="top",
                      y=0.33,
                      xanchor="left",
                      x=0.02))
## THIS IS TO BE ADDED IF STANDALONE VIZ, we are placing in HTML, this is not necessary

# TITLE
# fig.add_annotation(text = "Non-NYC Residents' Trips to NYC<br>by Home Municipality",
#                   align = 'left', x = 0.01, y = 0.95, showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   font = dict(family = "Arial", color = 'white', size = 19))

# SUBITILE
# fig.add_annotation(text = 'January 1, 2020 to October 31, 2020',
#                   align = 'left', x = 0.01, y = 0.86, showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   font = dict(family = "Arial", color = 'white', size = 13))

# EXPLANATORY TEXT
fig.add_annotation(text = 'Bubble size = # of Trips',
                  align = 'right', x = 0.02, y = 0.01, showarrow=False,
                  bordercolor = None, bgcolor = None,
                  font = dict(family = 'Droid Sans', color = 'white', size = 12))
fig.add_annotation(text = 'Click on/off boroughs in legend to compare or isolate migratory patterns',
                  align = 'right', x = 0.02, y = 0.082, showarrow=False,
                  bordercolor = None, bgcolor = None,
                  font = dict(family = 'Droid Sans', color = 'white', size = 12))
# speed this viz up so it doesn't look like crap..
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 80 
# save the HTML, send it on over
fig.write_html(r'Maps/html_maps/subplace_to_nyc.html',auto_open = True,auto_play = False, include_plotlyjs = 'cdn')